# Dogs-vs-Cats - Kaggle


In this project, you'll write an algorithm to classify whether images contain either a dog or a cat. We'll be using this dataset https://www.kaggle.com/c/dogs-vs-cats/data

In [20]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [1]:
# Import all required modules

import tensorflow as tf
import keras
from os import getcwd

from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import TensorBoard, ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import cv2
import numpy as np
from tqdm import tqdm
import os
from random import shuffle
from time import time
from matplotlib import pyplot
from matplotlib.image import imread

In [2]:
# link google drive where the dataset is placed
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#### Load the data
The dataset is available in two parts, training and testing.

In [9]:
train_path = '/content/drive/MyDrive/Colab Notebooks/Dogs-vs-Cats_Classifier/Dogs-Cats-Dataset/train_images'
test_path = '/content/drive/MyDrive/Colab Notebooks/Dogs-vs-Cats_Classifier/Dogs-Cats-Dataset/test_images'

#### Image Labeling

In [10]:
def get_label(imag):
    """
    Argument:
    imag - The name of the image whose label we want to get
    
    Return:
    respective label for cat or dog (cat = 0, dog = 1,)
    """
    term = imag.split('.')[0]
    if term == 'cat':
        return [0]
    else:
        return [1]

In [28]:
# Hyperparameters 
IMG_SIZE = 50
LR = 0.0003
BATCH_SIZE = 32

In [12]:
def get_training_data():
    """Returns the training data from train_path.
    Images are read in grayscale format and resized to IMG_SIZE dimension square.
    The whole data is saved with numpy in .npy format for quick loading for future purpose.
    """
    training_data = []
    if os.path.isfile('training_data_{}.npy'.format(IMG_SIZE)):
        return np.load('training_data_{}.npy'.format(IMG_SIZE))
    else:
        for img in tqdm(os.listdir(train_path)):
            label = get_label(img)
            path = os.path.join(train_path,img)
            img = cv2.resize(cv2.imread(path,cv2.IMREAD_GRAYSCALE), (IMG_SIZE,IMG_SIZE))
            img = img/255
            training_data.append([np.array(img),np.array(label)])
        shuffle(training_data)
        np.save('training_data_{}.npy'.format(IMG_SIZE),training_data)
        return np.array(training_data)

In [13]:
def get_testing_data():
    """Returns the testing data from test_path.
    Images are read in grayscale format and resized to IMG_SIZE dimension square.
    The whole data is saved with numpy in .npy format for quick loading for future purpose.
    """
    testing_data = []
    if os.path.isfile('testing_data_{}.npy'.format(IMG_SIZE)):
        return np.load('testing_data_{}.npy'.format(IMG_SIZE))
    else:
        for img in tqdm(os.listdir(test_path)):
            img_id = int(img.split('.')[0])
            path = os.path.join(test_path,img)
            img = cv2.resize(cv2.imread(path,cv2.IMREAD_GRAYSCALE), (IMG_SIZE,IMG_SIZE))
            img = img/255
            testing_data.append([np.array(img),img_id])
        testing_data.sort(key = lambda x: x[1])
        np.save('testing_data_{}.npy'.format(IMG_SIZE),testing_data)
        return np.array(testing_data)

#### Data Preparation

In [14]:
data = get_training_data()

partition = 1000             # Breaking -ve index
train = data[:-partition]    # For Training purpose
test= data[-partition:]      # For Validation purpose

# Training set
X_train = np.array([i[0] for i in train]).reshape(-1,IMG_SIZE,IMG_SIZE,1)
y_train = np.array([i[1] for i in train])

# Validation set
X_val = np.array([i[0] for i in test]).reshape(-1,IMG_SIZE,IMG_SIZE,1)
y_val = np.array([i[1] for i in test])

100%|██████████| 24254/24254 [1:41:33<00:00,  3.98it/s]


#### Image Augmentation for better results

In [15]:
"""Effects added on image
    Rotation - ± 50 deegrees,
    Width Shift - ± 15 %
    Height Shift - ± 15 %
    Zoom - 30%
    Horizontal Flip
    Vertical Flip
"""
datagen = ImageDataGenerator(rotation_range=20,width_shift_range=0.05,height_shift_range=0.05,
                            zoom_range=0.05,horizontal_flip=True,vertical_flip=False)

# Calculation of necessary internal data for all images.
datagen.fit(X_train)

### NN Model definition

In [25]:
def get_model(saved=True):
    """This method returns the model used.
    Returns a saved model if MODEL_NAME is found.
    CovNet Architecture
    
    Arguments:
    saved - Get the saved model from the MODEL_PATH if exists.(default True)
    
    Returns:
    model - The complete uncompiled Keras model.
    """
    # tf.reset_default_graph()
    
    if os.path.isfile(MODEL_PATH) and saved :
        print("Loading saved model {}".format(MODEL_NAME))
        return load_model(MODEL_PATH)
    
    # Declaring model
    model = Sequential()

    # 1st Block
    model.add(Conv2D(input_shape=(IMG_SIZE, IMG_SIZE, 1),filters=128, kernel_size=5, strides=1,padding='same',name = 'blk1_conv1'))
    model.add(Conv2D(filters=128, kernel_size=5, strides=1,padding='same',name = 'blk1_conv2'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=2,name = 'blk1_mxPool'))

    # 2nd Block
    model.add(Conv2D(filters=64, kernel_size=5, strides=1,padding='same',name = 'blk2_conv1'))
    model.add(Conv2D(filters=64, kernel_size=5, strides=1,padding='same',name = 'blk2_conv2'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=2,name = 'blk2_mxPool'))
    
    # 3rd Block
    model.add(Conv2D(filters=32, kernel_size=5, strides=1,padding='same',name = 'blk3_conv1'))
    model.add(Conv2D(filters=32, kernel_size=5, strides=1,padding='same',name = 'blk3_conv2'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=2,name = 'blk3_mxPool'))

    # 4th Block - FC Block
    dr_rate = 0.35
    model.add(Flatten(name = 'blk4_flatten'))
    model.add(Dropout(dr_rate,name = 'blk4_droupout1'))
    model.add(Dense(512, activation='relu',name = 'blk4_dense1'))
    model.add(Dropout(dr_rate,name = 'blk4_droupout2'))
    model.add(Dense(128, activation='relu',name = 'blk4_dense2'))
    model.add(Dropout(dr_rate,name = 'blk4_droupout3'))
    model.add(Dense(1, activation='sigmoid',name = 'blk4_dense3'))

    return model

In [26]:
model = get_model()
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
blk1_conv1 (Conv2D)          (None, 50, 50, 128)       3328      
_________________________________________________________________
blk1_conv2 (Conv2D)          (None, 50, 50, 128)       409728    
_________________________________________________________________
blk1_mxPool (MaxPooling2D)   (None, 25, 25, 128)       0         
_________________________________________________________________
blk2_conv1 (Conv2D)          (None, 25, 25, 64)        204864    
_________________________________________________________________
blk2_conv2 (Conv2D)          (None, 25, 25, 64)        102464    
_________________________________________________________________
blk2_mxPool (MaxPooling2D)   (None, 12, 12, 64)        0         
_________________________________________________________________
blk3_conv1 (Conv2D)          (None, 12, 12, 32)        5

In [32]:
# Optimizer (Adam Optimizer)
adam = Adam(lr = LR)

# Callbacks Declared
tensorboard = TensorBoard(log_dir="logs/{}".format(time()),batch_size=BATCH_SIZE)
       #Supported in new version of keras ,update_freq='epoch')
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.3,patience=3,verbose=1,
                              mode='max', min_lr=0.000001)
early_stop = EarlyStopping(monitor='val_loss',patience=3,verbose=1,mode='min')
      #Supported in new version of keras ,restore_best_weights=True)
model_checkpoint = ModelCheckpoint(filepath=MODEL_PATH,monitor='val_accuracy',verbose=1,save_best_only=True,
                                  mode='max',period=3)

model.compile(optimizer = adam,loss='binary_crossentropy',metrics=['accuracy'])

### Training

In [37]:
generator_train = True
EPOCHS = 30
callbacks=[tensorboard,reduce_lr,early_stop,model_checkpoint]

if generator_train:
    print(f'Training model {MODEL_NAME} using Image Augmentation')
    hist = model.fit_generator(datagen.flow(X_train,y_train,batch_size=BATCH_SIZE),
                               steps_per_epoch=len(X_train)//BATCH_SIZE,epochs=EPOCHS,verbose=2,
                               validation_data=(X_val,y_val),callbacks=callbacks)
else:
    print(f'Training model {MODEL_NAME} using normal image data provided')
    hist = model.fit(X_train,y_train,epochs=EPOCHS,batch_size=BATCH_SIZE,validation_data=(X_val,y_val),
                     verbose=2,callbacks=callbacks)
# model.save(MODEL_PATH)     

Training model dogs_cats_LR-0.0003_MODEL-CovNet-128(2)-64(2)-32(2)-512-128-1.h5 using Image Augmentation
Epoch 1/30
726/726 - 18s - loss: 0.3233 - accuracy: 0.8571 - val_loss: 0.3343 - val_accuracy: 0.8470
Epoch 2/30

Epoch 00002: val_accuracy did not improve from 0.85600
726/726 - 18s - loss: 0.3226 - accuracy: 0.8599 - val_loss: 0.3316 - val_accuracy: 0.8520
Epoch 3/30
726/726 - 18s - loss: 0.3279 - accuracy: 0.8574 - val_loss: 0.3216 - val_accuracy: 0.8550
Epoch 4/30
726/726 - 17s - loss: 0.3230 - accuracy: 0.8591 - val_loss: 0.3267 - val_accuracy: 0.8550
Epoch 5/30

Epoch 00005: val_accuracy did not improve from 0.85600
726/726 - 17s - loss: 0.3249 - accuracy: 0.8552 - val_loss: 0.3269 - val_accuracy: 0.8550
Epoch 6/30

Epoch 00006: ReduceLROnPlateau reducing learning rate to 1e-06.
726/726 - 18s - loss: 0.3255 - accuracy: 0.8563 - val_loss: 0.3283 - val_accuracy: 0.8540
Epoch 00006: early stopping


### Testing

In [39]:
test_data = get_testing_data()

100%|██████████| 12500/12500 [52:43<00:00,  3.95it/s]


In [40]:
X_test = np.array([i[0] for i in test_data]).reshape(-1,IMG_SIZE,IMG_SIZE,1)
ids = [i[1] for i in test_data]

In [41]:
pred = model.predict(X_test)

In [42]:
filename = 'submission-{}.csv'.format(time())

with open(filename,'w') as f:
    f.write('id,label\n')
with open(filename,'a') as f:
    for i in range(len(X_test)):
        f.write('{},{}\n'.format(ids[i],pred[i][0]))